<a href="https://colab.research.google.com/github/nayem9b/Bash-scripting/blob/main/Lets_Build_From_Ashes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
from glob import glob
import matplotlib.pyplot as plt
import random
import cv2
import pandas as pd
import numpy as np
import matplotlib.gridspec as gridspec
import seaborn as sns
import zlib
import itertools
import sklearn
import itertools
import scipy
import skimage
from skimage.transform import resize
import csv
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
from sklearn import model_selection
from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix, make_scorer, accuracy_score, classification_report
import keras
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Lambda, MaxPool2D, BatchNormalization

#from keras.preprocessing.image import ImageDataGenerator
from keras import models, layers, optimizers
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.utils import class_weight
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta, RMSprop
from keras.models import Sequential, model_from_json
from keras.layers import Activation,Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.layers import MaxPooling2D,AveragePooling2D, GlobalAveragePooling2D,BatchNormalization
from keras.utils import array_to_img, img_to_array, load_img
#from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
%matplotlib inline

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
normal_dir = '/content/drive/MyDrive/all_denoised_document/Speckle Noise/Noised/CNV'
dme_dir = '/content/drive/MyDrive/all_denoised_document/Speckle Noise/Noised/DME'
drusen_dir = '/content/drive/MyDrive/all_denoised_document/Speckle Noise/Noised/DRUSEN'
cnv_dir = '/content/drive/MyDrive/all_denoised_document/Speckle Noise/Noised/NORMAL'
# Image dimensions
image_size = (256, 256)
num_classes = 4  # NORMAL and DME

In [6]:
def load_and_preprocess_images(image_dir, label):
    images = []
    labels = []
    for filename in os.listdir(image_dir):
        if filename.endswith('.png'):
            img = load_img(os.path.join(image_dir, filename), target_size=image_size)
            img_array = img_to_array(img)
            img_array /= 255.0  # Normalize pixel values
            images.append(img_array)
            labels.append(label)
    return images, labels

In [7]:
normal_images, normal_labels = load_and_preprocess_images(normal_dir, label='NORMAL')
dme_images, dme_labels = load_and_preprocess_images(dme_dir, label='DME')
cnv_images, cnv_labels = load_and_preprocess_images(cnv_dir, label='CNV')
drusen_images, drusen_labels = load_and_preprocess_images(drusen_dir, label='DRUSEN')

In [8]:
print("Normal_images: ", len(normal_images))
print("DME_images: ", len(dme_images))
print("CNV_images: ", len(cnv_images))
print("Drusen_images: ", len(drusen_images))

Normal_images:  1000
DME_images:  1000
CNV_images:  1000
Drusen_images:  1000


In [9]:
# Combine datasets
X = np.array(normal_images + dme_images + cnv_images + drusen_images)
y = np.array(['NORMAL'] * len(normal_images) + ['DME'] * len(dme_images) +
             ['CNV'] * len(cnv_images) + ['DRUSEN'] * len(drusen_images))

In [10]:
# Convert labels to numeric values
class_names = ['NORMAL', 'DME', 'CNV', 'DRUSEN']
y_numeric = np.array([class_names.index(label) for label in y])

In [11]:
# One-hot encode labels
y_encoded = to_categorical(y_numeric, num_classes=len(class_names))

# Split dataset into training (80%) and testing (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_numeric)

print("Training samples:", len(X_train))
print("Testing samples:", len(X_test))

Training samples: 3200
Testing samples: 800


In [12]:
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, GlobalAveragePooling2D, Multiply, Reshape

# Efficient Channel Attention (ECA) block
class ECABlock(tf.keras.layers.Layer):
    def __init__(self, kernel_size=3):
        super(ECABlock, self).__init__()
        self.kernel_size = kernel_size

    def call(self, inputs):
        # Global Average Pooling (GAP) -> Shape: (batch_size, 1280)
        avg_pool = GlobalAveragePooling2D()(inputs)

        # Expand dimensions for Conv1D (batch_size, 1280, 1)
        avg_pool = tf.expand_dims(avg_pool, axis=-1)

        # Apply Conv1D
        conv1d = Conv1D(1, kernel_size=self.kernel_size, padding="same", activation="sigmoid")(avg_pool)

        # Reshape to match original input dimensions
        conv1d = Reshape((1, 1, inputs.shape[-1]))(conv1d)  # (batch_size, 1, 1, 1280)

        # Multiply attention weights with input feature maps
        output = Multiply()([inputs, conv1d])

        return output


In [13]:
def build_model():
    base_model = tf.keras.applications.EfficientNetB0(include_top=False, weights="imagenet", input_shape=(224, 224, 3))

    x = base_model.output  # Shape: (batch_size, 7, 7, 1280)

    x = ECABlock()(x)  # Apply ECA block

    x = GlobalAveragePooling2D()(x)  # Ensure it's a KerasTensor

    x = tf.keras.layers.Dense(256, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(4, activation="softmax")(x)  # 4 classes: CNV, DME, DRUSEN, NORMAL

    model = tf.keras.Model(inputs=base_model.input, outputs=x)

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Create and summarize the model
model = build_model()
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling (Rescaling)     │ (None, 224, 224, 3)    │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ normalization             │ (None, 224, 224, 3)    │              7 │ rescaling[0][0]        │
│ (Normalization)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling_1 (Rescaling)   │ (None, 224, 224, 3)    │              0 │ normalization[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv_pad             │ (None, 225, 225, 3)    │              0 │ rescaling_1[0][0]      │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv (Conv2D)        │ (None, 112, 112, 32)   │            864 │ stem_conv_pad[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_bn                   │ (None, 112, 112, 32)   │            128 │ stem_conv[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_activation           │ (None, 112, 112, 32)   │              0 │ stem_bn[0][0]          │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_dwconv            │ (None, 112, 112, 32)   │            288 │ stem_activation[0][0]  │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_bn                │ (None, 112, 112, 32)   │            128 │ block1a_dwconv[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_activation        │ (None, 112, 112, 32)   │              0 │ block1a_bn[0][0]       │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_squeeze        │ (None, 32)             │              0 │ block1a_activation[0]… │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reshape        │ (None, 1, 1, 32)       │              0 │ block1a_se_squeeze[0]… │
│ (Reshape)                 │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reduce         │ (None, 1, 1, 8)        │            264 │ block1a_se_reshape[0]… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_expand    

 Total params: 4,378,535 (16.70 MB)

 Trainable params: 4,336,512 (16.54 MB)

 Non-trainable params: 42,023 (164.16 KB)

In [1]:
epochs = 20
batch_size = 32

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=epochs,
    batch_size=batch_size
)


NameError: name 'model' is not defined